# Grab correlations by tissue

The documentation is available at http://http://isbtranslatorapi.adversary.us/

In [1]:
import urllib2
import json
import requests
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s %(message)s',)

In [2]:
base_url = 'http://isbtranslatorapi.adversary.us'

In [16]:
# The base metadata takes a minute.  It is counting a lot of stuff
def query_isb(endpoint, data={}, base_url=base_url):
    req = requests.post('%s/%s' % (base_url,endpoint), data=data)
    return req.json()

def get_analytes(kwargs):
    kw_local = kwargs.copy()
    frm = 0
    size = 10000
    meta = []
    kw_local['from'] = frm
    kw_local['size'] = size
    res = query_isb('/v1/analyte', data=kw_local)
    meta += res
    # Note: this is relying on the pagination, it would be smarter to just partition
    # the *sig_ids* set which would greatly speed up the query
    while len(res) > 0:
        kw_local['from'] += size
        frm = kw_local['from']
        logging.debug("Saving records from %i to %i" %(frm, frm+size))
        res = query_isb('/v1/analyte', data=kw_local)
        meta+=res
    return meta

def get_correlations(kwargs):
    kw_local = kwargs.copy()
    sigs = []
    frm = 0
    size = 10000
    meta = []
    kw_local['from'] = frm
    kw_local['size'] = size
    res = query_isb('v1/correlation', data=kw_local)
    correlations = res[:]
    while len(res) > 0:
        logging.debug("Saving records from %i to %i" %(frm, frm+size))
        kw_local['from'] += size
        frm = kw_local['from']
        res = query_isb('v1/correlation', data=kw_local)
        correlations += res
    return correlations

def get_correlations_undumb(kwargs, idlist=[]):
    kw_local = kwargs.copy()
    sigs = []
    frm = 0
    size = 50
    corr = []
    sub_ids = idlist[frm:frm+size]
    kw_local['ids1'] = ','.join(sub_ids)
    kw_local['size'] = size*size 
    prev = 0
    corr += query_isb('v1/correlation', data=kw_local)
    while len(sub_ids) > 0:
        frm += size
        sub_ids = idlist[frm:frm+size]
        kw_local = kwargs.copy()
        kw_local['ids1'] = ','.join(sub_ids)
        kw_local['size'] = size*size 
        corr += query_isb('v1/correlation', data=kw_local)
        curr = len(corr) - prev
        logging.debug("%i records"  % (len(corr,)))
        prev = curr

## Metadata access

How do I find the different things I am interested in?

In [4]:
print "You can view see what values are associated with a field"
print "Note most queries are paginated using from and size."

print json.dumps( query_isb('v1/metadata/analytes', data={'from':0, 'size':50}), indent=2)


2017-05-10 16:07:16,350 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us


You can view see what values are associated with a field
Note most queries are paginated using from and size.


2017-05-10 16:07:24,048 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/metadata/analytes HTTP/1.1" 200 454


{
  "fields": [
    "type", 
    "phylum", 
    "vendor", 
    "pubmed_id", 
    "hmdb", 
    "hgnc_id", 
    "gene_location", 
    "_id", 
    "aggregation_level", 
    "description", 
    "uniprot", 
    "tissue_id", 
    "tissue_description", 
    "sub_pathway", 
    "order", 
    "subsystem", 
    "pubchem", 
    "tissue", 
    "nsamples", 
    "health_status", 
    "name", 
    "abbreviation", 
    "kegg", 
    "kingdom", 
    "super_pathway", 
    "study_title", 
    "class", 
    "source_project", 
    "measurement_technology", 
    "family", 
    "category", 
    "peptide"
  ], 
  "Unique Analytes": 989596
}


In [5]:
print "You can view see what values are associated with a field"
print "Note most queries are paginated using from and size."

print json.dumps( query_isb('v1/metadata/analytes/tissue', data={'from':0, 'size':33}), indent=2)


2017-05-10 16:07:24,057 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us


You can view see what values are associated with a field
Note most queries are paginated using from and size.


2017-05-10 16:07:27,667 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/metadata/analytes/tissue HTTP/1.1" 200 405


{
  "count": 33, 
  "tissue": [
    "Lung", 
    "Testis", 
    "Cervix", 
    "Uterus", 
    "Bladder", 
    "Liver", 
    "Kidney", 
    "Spleen", 
    "Eye", 
    "Nerve", 
    "Connective Tissue", 
    "Stomach", 
    "Blood Vessel", 
    "Head and Neck", 
    "Esophagus", 
    "B-cells", 
    "Breast", 
    "Pituitary", 
    "Heart", 
    "Mesothelium", 
    "Ovaries", 
    "Bile Duct", 
    "Thyroid", 
    "Skin", 
    "Prostate", 
    "Brain", 
    "Muscle", 
    "Thymus", 
    "Blood", 
    "Colon", 
    "Adipose Tissue", 
    "Adrenal Gland", 
    "Pancreas"
  ]
}


In [35]:


corr_nets = {}
fanc_genes = [x.strip() for x in 'BRCA1, BRCA2, BRIP1, ERCC4, FANCA, FANCB, FANCC, FANCD2, FANCE, FANCF, FANCG, FANCI, FANCL, FANCM, PALB2, RAD51, RAD51C, UBE2T, XRCC2'.split(',')]
tissues = ["Skin", "Kidney","Heart"]
genes = {}
for tis in tissues:
    kwargs = {'category':"gene expression", 'tissue':tis, 'source_project':'GTEx'}
    t_anal = get_analytes(kwargs)  
    genes[tis] = t_anal
    anal_ids = [t['_id'] for t in t_anal if t['abbreviation'] in fanc_genes]
    corr = get_correlations({'ids1': ','.join(anal_ids)})
    corr_nets[tis] = pandas.DataFrame(corr)


2017-05-10 17:53:53,240 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:54:15,223 DEBUG http://isbtranslatorapi.adversary.us:80 "POST //v1/analyte HTTP/1.1" 200 3451834
2017-05-10 17:54:16,443 DEBUG Saving records from 10000 to 20000
2017-05-10 17:54:16,445 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:54:36,840 DEBUG http://isbtranslatorapi.adversary.us:80 "POST //v1/analyte HTTP/1.1" 200 2866798
2017-05-10 17:54:37,850 DEBUG Saving records from 20000 to 30000
2017-05-10 17:54:37,853 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:54:50,178 DEBUG http://isbtranslatorapi.adversary.us:80 "POST //v1/analyte HTTP/1.1" 200 3
2017-05-10 17:54:50,259 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:54:57,278 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2260564
2017-05-10 17:54:58,325 DEBUG Saving records from 0 t

2017-05-10 17:57:11,687 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2263658
2017-05-10 17:57:12,635 DEBUG Saving records from 270000 to 280000
2017-05-10 17:57:12,637 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:57:16,968 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2260020
2017-05-10 17:57:17,931 DEBUG Saving records from 280000 to 290000
2017-05-10 17:57:17,933 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:57:22,304 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2262969
2017-05-10 17:57:23,257 DEBUG Saving records from 290000 to 300000
2017-05-10 17:57:23,260 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 17:57:27,675 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2266488
2017-05-10 17:57:28,636 DEBUG Saving records from 300000 to 3

2017-05-10 18:00:28,867 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2272543
2017-05-10 18:00:29,808 DEBUG Saving records from 170000 to 180000
2017-05-10 18:00:29,810 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 18:00:34,150 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2272048
2017-05-10 18:00:35,113 DEBUG Saving records from 180000 to 190000
2017-05-10 18:00:35,115 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 18:00:39,476 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2268116
2017-05-10 18:00:40,405 DEBUG Saving records from 190000 to 200000
2017-05-10 18:00:40,408 DEBUG Starting new HTTP connection (1): isbtranslatorapi.adversary.us
2017-05-10 18:00:44,832 DEBUG http://isbtranslatorapi.adversary.us:80 "POST /v1/correlation HTTP/1.1" 200 2271406
2017-05-10 18:00:45,872 DEBUG Saving records from 200000 to 2

In [39]:
len(corr_nets['Heart'][corr_nets['Heart'].pvalue < .05/len(corr_nets['Heart'].pvalue)])

158152